In [140]:
import numpy as np
from typing import Tuple, Union

# Direct Method for Solving Linear Systems

## Chapter 6.1 Linear Systems of Equations

### Backward and Forward Substituition 
Given an augmented upper triangular matrix $U$ with the shape $n\times (n + 1)$, we can compute the solutions for the systems by formula below:
$$x_n = \frac{u_{n, n+1}}{u_{n, n}}$$
$$x_i=\frac{1}{u_{i, i}}\biggl(u_{i, n+1}-\sum^{n}_{j=i+1}u_{i, j}x_{j}\biggr)$$
where $i=1,2,\cdots,n-1$

Given an augmented lower triangular matrix $L$ with the shape $n\times (n + 1)$, we can compute the solutions for the systems by formula below:
$$x_1 = \frac{l_{1, n+1}}{l_{1, 1}}$$
$$x_i=\frac{1}{l_{i, i}}\biggl(l_{i, n+1}-\sum^{i - 1}_{j=1}l_{i, j}x_{j}\biggr)$$


In [141]:
def backSub(U:np.ndarray) -> np.ndarray:
    N, M = U.shape
    x = np.zeros((N, 1), dtype=np.float64)
    x[N - 1][0] = U[N - 1][M - 1] / U[N - 1][N - 1]    
    for jdx in range(N - 2, -1, -1):
        x[jdx] = (U[jdx][M - 1] - np.sum(U[jdx, jdx + 1:N] @ x[jdx + 1:])) / U[jdx][jdx]
    return x

def forSub(L:np.ndarray) -> np.ndarray:
    N, M = L.shape
    x = np.zeros((N, 1), dtype=np.float64)
    x[0][0] = L[0][M - 1] / L[0][0]   
    for jdx in range(1, N):
        x[jdx] = (L[jdx][M - 1] - np.sum(L[jdx, :jdx] @ x[:jdx])) / L[jdx][jdx]
    return x

In [142]:
def forwardElimination(A:np.ndarray, return_A:bool=False) -> Union[Tuple[np.ndarray, np.ndarray], np.ndarray]:
    '''
    Parameters:
    A: Augmented matrix of the system of equations Ax = b
    '''
    N, M = A.shape
    A_ = np.copy(A).astype(float)
    for idx in range(N - 1):
        non_zero = np.nonzero(A_[idx:, idx])
        if not non_zero[0].size:
            raise ValueError("No Unique Solution Exists...")
        ptr = non_zero[0][0] + idx
        A_[[idx, ptr]] = A_[[ptr, idx]]
            
        for jdx in range(idx + 1, N):
            m = A_[jdx][idx] / A_[idx][idx]
            A_[jdx] -= m * A_[idx]
            
    if A_[N - 1][N - 1] == 0:
        raise ValueError("No Unique Solution Exists...")
    return backSub(A_) if not return_A else (A_, backSub(A_))

def backwardElimination(A:np.ndarray, return_A:bool=False) -> Union[Tuple[np.ndarray, np.ndarray], np.ndarray]:
    '''
    Parameters:
    A: Augmented matrix of the system of equations Ax = b
    '''
    N, M = A.shape
    A_ = np.copy(A).astype(float)
    for idx in range(N - 1, 0, -1):
        non_zero = np.nonzero(A_[:idx, idx])
        if not non_zero[0].size:
            raise ValueError("No Unique Solution Exists...")
        ptr = non_zero[0][::-1][0]
        A_[[idx, ptr]] = A_[[ptr, idx]]
            
        for jdx in range(idx - 1, -1, -1):
            m = A_[jdx][idx] / A_[idx][idx]
            A_[jdx] -= m * A_[idx]
            
    if A_[0][0] == 0:
        raise ValueError("No Unique Solution Exists...")
    return forSub(A_) if not return_A else (A_, forSub(A_))


In [143]:
A = np.array([
   [1, -1, 2, -1, -8],
   [2, -2, 3, -3, -20],
   [1, 1, 1, 0, -2],
   [1, -1, 4, 3, 4] 
], dtype=np.float32)

forwardElimination(A), backwardElimination(A), np.linalg.solve(A[:, :4], A[:, -1])

(array([[-7.],
        [ 3.],
        [ 2.],
        [ 2.]]),
 array([[-7.],
        [ 3.],
        [ 2.],
        [ 2.]]),
 array([-7.,  3.,  2.,  2.], dtype=float32))

In [144]:
A = np.random.randint(-50, 50, 42).reshape(6, 7)
print(A)
forwardElimination(A), backwardElimination(A), np.linalg.solve(A[:, :6], A[:, -1])

[[-16  39 -49   5   1 -33 -37]
 [-14  15  27  20   8  12   1]
 [-37  41 -18  40  29 -37 -43]
 [-19  15 -41 -16   2   0   3]
 [ 23  49 -33 -38   4  28  47]
 [-16  29  35   6 -36 -16 -23]]


(array([[-0.01384319],
        [ 0.07793067],
        [ 0.46375197],
        [-1.18712871],
        [ 0.79389547],
        [ 0.37561189]]),
 array([[-0.01384319],
        [ 0.07793067],
        [ 0.46375197],
        [-1.18712871],
        [ 0.79389547],
        [ 0.37561189]]),
 array([-0.01384319,  0.07793067,  0.46375197, -1.18712871,  0.79389547,
         0.37561189]))

## Chapter 6.3 Linear Algebra and Matrix Inversion
Consider
$$
A = 
\begin{bmatrix}
a_{11} & a_{12} & a_{13} & \cdots & a_{1n}\\
\vdots & \vdots & \vdots & \ddots & \vdots\\
a_{n1} & a_{n2} & a_{n3} & \cdots & a_{nn}\\
\end{bmatrix}\,\,\,\,\,

B= 
\begin{bmatrix}
b_{11} & b_{12} & b_{13} & \cdots & b_{1n}\\
\vdots & \vdots & \vdots & \ddots & \vdots\\
b_{n1} & b_{n2} & b_{n3} & \cdots & b_{nn}\\
\end{bmatrix}
$$

By the definition of inverse of a matrix, we have $AB=I$, which means

$$
AB = 
\begin{bmatrix}
\sum_{k=1}^{n} a_{1k}b_{k1} & \sum_{k=1}^{n} a_{1k}b_{k2} & \cdots & \sum_{k=1}^{n} a_{1k}b_{kn} \\
\vdots & \vdots & \ddots & \vdots \\
\sum_{k=1}^{n} a_{nk}b_{k1} & \sum_{k=1}^{n} a_{nk}b_{k2} & \cdots & \sum_{k=1}^{n} a_{nk}b_{kn} \\
\end{bmatrix}=
\begin{bmatrix}
1 & 0 & \cdots & 0 \\
0 & 1 & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & 1 \\
\end{bmatrix}
$$

Since the coefficient of unknown $b_{ij}$ for fixed $j$ and each $i=1, 2, \cdots n$ are the same we can actually view in such way

$$
A'=AB_j=I_n = \begin{bmatrix}
0 \\
\vdots \\
0 \\
1 \\
0 \\
\vdots \\
0
\end{bmatrix}
$$

where $B_j$ represents the $j$ th column of matrix $B$. 

Now, we can treat it as a system of linear equation $Ax=b$ where the $B_j$ is now our $x$ and the corresponding $j$ th columun of identity matrix is now our $b$. So just perform backward substitution and we get the values of $B_j$

In [145]:
def Inverse(A:np.ndarray) -> np.ndarray:
    N, M = A.shape
    I = np.eye(N, N)
    A_ = np.copy(A).astype(float)
    for idx in range(N):
        non_zero = np.nonzero(A_[idx:, idx])
        if not non_zero[0].size:
            raise ValueError("The matrix is singular")
        ptr = non_zero[0][0] + idx
        A_[[idx, ptr]] = A_[[ptr, idx]]
        I[[idx, ptr]] = I[[ptr, idx]]

        for jdx in range(idx + 1, N):
            m = A_[jdx][idx] / A_[idx][idx]
            A_[jdx] -= m * A_[idx]
            I[jdx] -= m * I[idx]

    if A_[N - 1][N - 1] == 0:
        raise ValueError("The matrix is singular")
    
    A_inv = np.eye(N, N)
    for idx in range(N):
        # Reshape B[:, idx] from row vector to column vector and stack with A_ since
        # the backSub function accepts the augmented matrix.
        # Then flatten the result from backSub to 1D since the returned value
        # will be 2D with shape (N, 1)
        A_inv[:, idx] = backSub(np.hstack((A_, I[:, idx].reshape((N, 1))))).flatten()
    return A_inv


In [146]:
# Illustration
A = np.array([
    [1, 2, -1],
    [2, 1, 0],
    [-1, 1, 2]
    ])
print("Inverse of A : ", Inverse(A))
A @ Inverse(A)


Inverse of A :  [[-0.22222222  0.55555556 -0.11111111]
 [ 0.44444444 -0.11111111  0.22222222]
 [-0.33333333  0.33333333  0.33333333]]


array([[ 1.00000000e+00,  5.55111512e-17,  0.00000000e+00],
       [-1.66533454e-16,  1.00000000e+00,  0.00000000e+00],
       [ 1.11022302e-16, -1.11022302e-16,  1.00000000e+00]])

## Chapter 6.4 The Determinant of a Matrix
> If B is a matrix obtained by multiplying a row of A by some non-zero constant ß, then
> - det(B) = ß * det(A)
> 
> In other words, you can essentially 'factor out' a constant from a row by just pulling it out front of the determinant.
> If B is a matrix obtained by swapping two rows of A, then
> 
> - det(B) = -det(A)
> 
> If you swap rows, flip the sign.
> 
> If B is a matrix obtained by adding a multiple of one row to another row in A, then
> 
> - det(B) = det(A)
> 
> The determinant doesn't change.

Reference:[https://stackoverflow.com/questions/2435133/what-is-the-best-algorithm-to-find-a-determinant-of-a-matrix](https://stackoverflow.com/questions/2435133/what-is-the-best-algorithm-to-find-a-determinant-of-a-matrix)

The algorithm below implemented using the property that the determinant of the diagonal or triangular  matrix no matter lower or upper, the determinant is equal to $0$

So by using this property and the rule $(2\text{ and }3)$ above we come to the conclusion that $det(A)=(-1)^{m}\prod_{i}u_{i, i}$, where $u$ is the entry of the upper triangular matrix after reduce matrix $A$ to row echelon form and $m$ is the number of time we interchange the row.

In [147]:
def det(A:np.ndarray) -> float:
    N, M = A.shape
    A_ = np.copy(A).astype(float)
    p = 0
    for idx in range(N - 1):
        non_zero = np.nonzero(A_[idx:, idx])
        if not non_zero[0].size:
            raise ValueError("No Unique Solution Exists...")
        ptr = non_zero[0][0] + idx

        # Row interchange causes sign of determinant flips
        if ptr != idx:
            A_[[idx, ptr]] = A_[[ptr, idx]]
            p += 1
            
        # Addition between row does not change the determinant
        for jdx in range(idx + 1, N):
            m = A_[jdx][idx] / A_[idx][idx]
            A_[jdx] -= m * A_[idx]
            
    if A_[N - 1][N - 1] == 0:
        raise ValueError("No Unique Solution Exists...")
    
    return np.prod(np.diag(A_)) * (-1) ** p

In [148]:
A = np.array([
   [1, -1, 2, -1],
   [2, -2, 3, -3],
   [1, 1, 1, 0],
   [1, -1, 4, 3] 
], dtype=np.float32)
det(A), np.linalg.det(A)

(4.0, 4.0)

In [149]:
A = np.random.randint(1, 20, 16).reshape((4, 4))
det(A), np.linalg.det(A)

(-5616.0, -5616.0)

## Chapter 6.5 Matrix Factorization

### $LU$ and $PLU$ Decomposition

In [150]:
def LU(A:np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    '''
    Returns:
    L: The lower triangular matrix obtained with main diagonal elements = 1
    U: The upper triangular matrix obtained
    '''

    if A[0][0] == 0:
        raise ValueError("Factorization impossible")
    
    N = A.shape[0]
    L = np.eye(N)
    U = np.zeros_like(A, dtype=float)
    A_ = np.copy(A).astype(float)

    for idx in range(N):
        if A_[idx][idx] - (L[idx, :idx] @ U[idx, :idx]) == 0:
            raise ValueError("Factorization impossible")
        # When idx = 0, L[0, :0] and U[:0, 0] are both empty arrays and @ two empty arrays result in 0.0
        U[idx][idx] = A_[idx][idx] - (L[idx, :idx] @ U[:idx, idx])
        for jdx in range(idx + 1, N):
            U[idx][jdx] = (A_[idx][jdx] - L[idx, :idx] @ U[:idx, jdx]) / L[idx][idx]
            L[jdx][idx] = (A_[jdx][idx] - L[jdx, :idx] @ U[:idx, idx]) / U[idx][idx]
    return L, U

In [151]:
A = np.array([
    [1, 1, 0, 3],
    [2, 1, -1, 1],
    [3, -1, -1, 2],
    [-1, 2, 3, -1]
], dtype=float)
L, U = LU(A)
L @ U

array([[ 1.,  1.,  0.,  3.],
       [ 2.,  1., -1.,  1.],
       [ 3., -1., -1.,  2.],
       [-1.,  2.,  3., -1.]])

In [152]:
def PLU(A:np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    '''
    PA = LU
    This method derived from the Gaussian Elimnation process
    Returns:
    P: The transposed permutation matrix so that A = PLU
    L: The lower triangular matrix obtained with main diagonal elements = 1
    U: The upper triangular matrix obtained
    '''
    N = A.shape[0]
    P = np.eye(N)
    L = np.zeros_like(A, dtype=float)
    U = np.copy(A).astype(float)
    
    for idx in range(N - 1):
        pivot_row = np.argmax(np.abs(U[idx:, idx])) + idx
        # (E_i <--> E_j)
        if pivot_row != idx:
            P[[idx, pivot_row]] = P[[pivot_row, idx]]
            U[[idx, pivot_row]] = U[[pivot_row, idx]]
            L[[idx, pivot_row]] = L[[pivot_row, idx]]
        for jdx in range(idx + 1, N):
            factor = U[jdx, idx] / U[idx, idx]
            L[jdx, idx] = factor
            # (E_i + mE_j) --> (E_i)
            U[jdx, :] -= factor * U[idx, :]
    np.fill_diagonal(L, 1)
    return P.T, L, U

In [153]:
A = np.array([
    [0, 0, -1, 1],
    [1, 1, -1, 2],
    [-1, -1, 2, 0],
    [1, 2, 0, 2]
])
P, L, U = PLU(A)
P @ L @ U

array([[ 0.,  0., -1.,  1.],
       [ 1.,  1., -1.,  2.],
       [-1., -1.,  2.,  0.],
       [ 1.,  2.,  0.,  2.]])

### $LDL^T$ Decomposition amd its pivoting version
Pseudocode of pivoting version is not available in _Numerical Analysis by Richard L. Burden_ but _Matrix Computation by G.H. Golub_

In [154]:
def LDL(A:np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    '''
    A = LDLT
    Returns:
    L: The lower triangular matrix obtained with main diagonal elements = 1
    D The diagonal matrix obtained
    '''
    N = A.shape[0]
    L = np.eye(N)
    D = np.zeros(N)
    v = np.zeros(N)
    for idx in range(N):
        for jdx in range(idx):
            v[jdx] = L[idx, jdx] * D[jdx]
        D[idx] = A[idx][idx] - L[idx, :idx] @ v[:idx]
        for jdx in range(idx + 1, N):
            L[jdx][idx] = (A[jdx][idx] - L[jdx, :idx] @ v[:idx]) / D[idx]
    D_ = np.zeros((N, N))
    D_[np.diag_indices(N)] = D
    return L, D_

In [155]:
A = np.array([
    [4, -1, 1],
    [-1, 4.25, 2.75],
    [1, 2.75, 3.5]
])
L, D = LDL(A)

L @ D @ L.T

array([[ 4.  , -1.  ,  1.  ],
       [-1.  ,  4.25,  2.75],
       [ 1.  ,  2.75,  3.5 ]])

In [156]:
def PLDL(A:np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    '''
    PAPT = LDLT
    Returns:
    P: The transposed permutation matrix
    L: The lower triangular matrix obtained with main diagonal elements = 1
    D The diagonal matrix obtained
    '''
    N = A.shape[0]
    P = np.eye(N)
    A_ = np.copy(A).astype(float)

    for idx in range(N):
        piv = np.argmax(np.diag(A_[idx:, idx:])) + idx
        P[[idx, piv]] = P[[piv, idx]]
        A_[[idx, piv]] = A_[[piv, idx]]
        A_[:, [idx, piv]] = A_[:, [piv, idx]]
        alpha = A_[idx, idx]
        v = np.copy(A_[idx + 1:, idx])
        A_[idx + 1:, idx] = v / alpha
        A_[idx + 1:, idx + 1:] -= np.outer(v, v) / alpha

    D = np.zeros((N, N))
    D[np.diag_indices(N)] = np.diag(A_)
    L =np.tril(A_, k=-1) + np.eye(N)
    return P, L, D

In [157]:
A = np.array([
    [1, 2, 3],
    [2, 4, 5],
    [3, 5, 6]
])
P, L, D = PLDL(A)
P.T @ L @ D @ L.T @ P

array([[1., 2., 3.],
       [2., 4., 5.],
       [3., 5., 6.]])

### Cholesky Decoomposition

In [158]:
def cho(A:np.ndarray) -> np.ndarray:
    N = A.shape[0]
    L = np.zeros((N, N))
    for idx in range(N):
        L[idx][idx] = np.sqrt(A[idx][idx] - np.sum(L[idx, :idx] ** 2))
        for jdx in range(idx + 1, N):
            L[jdx][idx] = (A[jdx][idx] - L[jdx, :idx] @ L[idx, :idx]) / L[idx][idx]
    return L

In [159]:
A = np.array([
    [4, -1, 1],
    [-1, 4.25, 2.75],
    [1, 2.75, 3.5]
])
L = cho(A)
L @ L.T

array([[ 4.  , -1.  ,  1.  ],
       [-1.  ,  4.25,  2.75],
       [ 1.  ,  2.75,  3.5 ]])

### Crout Factorization for Tridiagonal Linear Systems

In [160]:
def solveTri(A:np.ndarray, return_LUz:bool=False
             ) -> Union[Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray], np.ndarray]:
    '''
    By this factorization, A = LU, then the linear system Ax = b can be solved
    in such manner, L(Ux) = b, define (Ux) = z, then solve Lz = b first followed by
    Ux = z
    Parameters:
    A: Augmented matrix of the tridiagonal system of equations Ax = b
    Returns:
    L: The lower triangular matrix obtained
    U: The upper triangular matrix obtained which the diagonal elements are all ones
    z: The solution of Ux = z
    x: The solution of Ax = b or Lz = b
    '''
    N = A.shape[0]
    L = np.zeros((N, N), dtype=float)
    U = np.eye(N, dtype=float)
    z = np.zeros(N, dtype=float)
    x = np.zeros(N, dtype=float)

    # Solving Lz = b
    L[0, 0] = A[0, 0]
    U[0, 1] = A[0, 1] / L[0, 0]
    z[0] = A[0, N] / L[0, 0]

    for idx in range(1, N - 1):
        L[idx, idx - 1] = A[idx, idx - 1]
        L[idx, idx] = A[idx, idx] - L[idx, idx - 1] * U[idx - 1, idx]
        U[idx, idx + 1] = A[idx, idx + 1] / L[idx, idx]
        z[idx] = (A[idx, N] - L[idx, idx - 1] * z[idx - 1]) / L[idx, idx]
    
    L[N - 1, N - 2] = A[N - 1, N - 2]
    L[N - 1, N - 1] = A[N - 1, N - 1] - L[N - 1, N - 2] * U[N - 2, N - 1]
    z[N - 1] = (A[N - 1, N] - L[N - 1, N - 2] * z[N - 2]) / L[N - 1, N - 1]
    x[N - 1] = z[N - 1]

    for idx in range(N - 2, -1, -1):
        x[idx] = z[idx] - U[idx, idx + 1] * x[idx + 1]
    
    return x if not return_LUz else (L, U, z, x)

In [161]:
A = np.array([
    [2, -1, 0, 0, 1],
    [-1, 2, -1, 0, 0],
    [0, -1, 2, -1, 0],
    [0, 0, -1, 2, 1]
])
solveTri(A, return_LUz=True)

(array([[ 2.        ,  0.        ,  0.        ,  0.        ],
        [-1.        ,  1.5       ,  0.        ,  0.        ],
        [ 0.        , -1.        ,  1.33333333,  0.        ],
        [ 0.        ,  0.        , -1.        ,  1.25      ]]),
 array([[ 1.        , -0.5       ,  0.        ,  0.        ],
        [ 0.        ,  1.        , -0.66666667,  0.        ],
        [ 0.        ,  0.        ,  1.        , -0.75      ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([0.5       , 0.33333333, 0.25      , 1.        ]),
 array([1., 1., 1., 1.]))

In [162]:
A =np.array([
    [.5, .25, 0, 0, .35],
    [.35, .8, .4, 0, .77],
    [0, .25, 1, .5, -.5],
    [0, 0, 1, -2, -2.25]
])
solveTri(A)

array([-0.09357798,  1.58715596, -1.16743119,  0.5412844 ])